# Applied Dash Plotly to visualize Covid-19 Dashboard

## Import Library

In [1]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

## Download data

In [2]:
url_confirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
url_deaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
url_recovered = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

confirmed = pd.read_csv(url_confirmed)
deaths = pd.read_csv(url_deaths)
recovered = pd.read_csv(url_recovered)

In [3]:
confirmed.sample(10)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
65,Guangxi,China,23.8298,108.7881,2,5,23,23,36,46,...,13371,13371,13371,13371,13371,13371,13371,13371,13371,13371
97,NaN,Costa Rica,9.7489,-83.7534,0,0,0,0,0,0,...,1199055,1204164,1204164,1204164,1204164,1204164,1204164,1204164,1209725,1209725
33,NaN,Bulgaria,42.7339,25.4858,0,0,0,0,0,0,...,1296853,1296853,1296913,1297006,1297037,1297062,1297243,1297358,1297451,1297523
112,NaN,El Salvador,13.7942,-88.8965,0,0,0,0,0,0,...,201785,201785,201785,201785,201785,201785,201785,201785,201785,201785
147,NaN,Iceland,64.9631,-19.0208,0,0,0,0,0,0,...,209093,209093,209093,209093,209093,209093,209093,209137,209137,209137
22,NaN,Barbados,13.1939,-59.5432,0,0,0,0,0,0,...,106645,106645,106645,106645,106645,106645,106645,106645,106645,106798
127,Saint Barthelemy,France,17.9000,-62.8333,0,0,0,0,0,0,...,5439,5439,5439,5439,5439,5439,5439,5439,5441,5441
193,NaN,Namibia,-22.9576,18.4904,0,0,0,0,0,0,...,171156,171156,171156,171156,171156,171156,171156,171156,171156,171156
88,Tibet,China,31.6927,88.0924,0,0,0,0,0,0,...,1647,1647,1647,1647,1647,1647,1647,1647,1647,1647
269,Falkland Islands (Malvinas),United Kingdom,-51.7963,-59.5236,0,0,0,0,0,0,...,1930,1930,1930,1930,1930,1930,1930,1930,1930,1930


## Preprocessing Data

### Melting data

In [4]:
confirmed_date = confirmed.columns[4:] 
confirmed_geo = confirmed.columns[:4]
total_confirmed = confirmed.melt(id_vars=confirmed_geo, value_vars= confirmed_date,value_name='Confirmed',var_name='Date')
# total_confirmed
deaths_date = deaths.columns[4:]
deaths_geo = deaths.columns[:4]
total_deaths = deaths.melt(id_vars=confirmed_geo, value_vars=deaths_date, value_name='Deaths',var_name='Date')

recovered_date = recovered.columns[4:]
recovered_geo = recovered.columns[:4]
total_recovered= recovered.melt(id_vars=recovered_geo, value_vars=recovered_date, value_name='Recovered',var_name='Date')

### Merging Data

In [5]:
covid_data = total_confirmed.merge(total_deaths, on=confirmed_geo.to_list() + ['Date'], how='left')
covid_data = covid_data.merge(total_recovered, on = recovered_geo.to_list() + ['Date'], how='left')

In [6]:
covid_data.sample(10)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
254955,NaN,Chad,15.454200,18.732200,6/22/22,7424,193,0.0
298230,Guernsey,United Kingdom,49.448196,-2.589490,11/18/22,32666,65,0.0
169513,NaN,Kenya,-0.023600,37.906200,8/30/21,235298,4720,0.0
195228,NaN,Israel,31.046051,34.851612,11/27/21,1341881,8184,0.0
234118,NaN,Bolivia,-16.290200,-63.588700,4/11/22,903477,21899,0.0
251380,NaN,South Africa,-30.559500,22.937500,6/9/22,3975062,101448,0.0
3995,NaN,Somalia,5.152149,46.199616,2/4/20,0,0,0.0
215587,NaN,Venezuela,6.423800,-66.589700,2/5/22,494835,5481,0.0
71997,NaN,Burundi,-3.373100,29.918900,9/27/20,485,1,472.0
170724,NaN,Paraguay,-23.442500,-58.443800,9/3/21,458799,15984,0.0


In [7]:
covid_data['Date'] = pd.to_datetime(covid_data['Date'])

/tmp/ipykernel_31095/3902955790.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  covid_data['Date'] = pd.to_datetime(covid_data['Date'])


In [8]:
covid_data.isna().sum()

Province/State    226314
Country/Region         0
Lat                 2286
Long                2286
Date                   0
Confirmed              0
Deaths                 0
Recovered          24003
dtype: int64

In [9]:
covid_data['Recovered'] = covid_data['Recovered'].fillna(0)

In [10]:
covid_data['Active'] = covid_data['Confirmed'] - covid_data['Deaths'] - covid_data['Recovered']

In [11]:
covid_data_1 = covid_data.groupby(['Date'])[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum().reset_index()
covid_data_1

,Date,Confirmed,Deaths,Recovered,Active
0,2020-01-22,557,17,30.0,510.0
1,2020-01-23,657,18,32.0,607.0
2,2020-01-24,944,26,39.0,879.0
3,2020-01-25,1437,42,42.0,1353.0
4,2020-01-26,2120,56,56.0,2008.0
...,...,...,...,...,...
1138,2023-03-05,676024901,6877749,0.0,669147152.0
1139,2023-03-06,676082941,6878115,0.0,669204826.0
1140,2023-03-07,676213378,6879038,0.0,669334340.0
1141,2023-03-08,676392824,6880483,0.0,669512341.0


In [12]:
covid_data_2 = covid_data.groupby(['Date', 'Country/Region'])[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum().reset_index()
covid_data_2

,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0.0,0.0
1,2020-01-22,Albania,0,0,0.0,0.0
2,2020-01-22,Algeria,0,0,0.0,0.0
3,2020-01-22,Andorra,0,0,0.0,0.0
4,2020-01-22,Angola,0,0,0.0,0.0
...,...,...,...,...,...,...
229738,2023-03-09,West Bank and Gaza,703228,5708,0.0,697520.0
229739,2023-03-09,Winter Olympics 2022,535,0,0.0,535.0
229740,2023-03-09,Yemen,11945,2159,0.0,9786.0
229741,2023-03-09,Zambia,343135,4057,0.0,339078.0


In [13]:
covid_data_dict = covid_data[['Country/Region', 'Lat', 'Long']]
list_locations = covid_data_dict.set_index('Country/Region')[['Lat', 'Long']].T.to_dict()

/tmp/ipykernel_31095/3770569221.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  list_locations = covid_data_dict.set_index('Country/Region')[['Lat', 'Long']].T.to_dict()


In [24]:
covid_data_2.groupby(['Date','Country/Region'])[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum().reset_index()

,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0.0,0.0
1,2020-01-22,Albania,0,0,0.0,0.0
2,2020-01-22,Algeria,0,0,0.0,0.0
3,2020-01-22,Andorra,0,0,0.0,0.0
4,2020-01-22,Angola,0,0,0.0,0.0
...,...,...,...,...,...,...
229738,2023-03-09,West Bank and Gaza,703228,5708,0.0,697520.0
229739,2023-03-09,Winter Olympics 2022,535,0,0.0,535.0
229740,2023-03-09,Yemen,11945,2159,0.0,9786.0
229741,2023-03-09,Zambia,343135,4057,0.0,339078.0


In [22]:
print(covid_data_2.groupby(['Date','Country/Region'])[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum().reset_index().info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229743 entries, 0 to 229742
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Date            229743 non-null  datetime64[ns]
 1   Country/Region  229743 non-null  object        
 2   Confirmed       229743 non-null  int64         
 3   Deaths          229743 non-null  int64         
 4   Recovered       229743 non-null  float64       
 5   Active          229743 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 10.5+ MB
None


In [23]:
covid_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229743 entries, 0 to 229742
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Date            229743 non-null  datetime64[ns]
 1   Country/Region  229743 non-null  object        
 2   Confirmed       229743 non-null  int64         
 3   Deaths          229743 non-null  int64         
 4   Recovered       229743 non-null  float64       
 5   Active          229743 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 10.5+ MB


In [ ]:
value_confirmed = covid_data_2[covid_data_2['Country/Region'] == w_countries]['confirmed'].iloc[-1] - covid_data_2[covid_data_2['Country/Region'] == w_countries]['confirmed'].iloc[-2]
delta_confirmed = covid_data_2[covid_data_2['Country/Region'] == w_countries]['confirmed'].iloc[-2] - covid_data_2[covid_data_2['Country/Region'] == w_countries]['confirmed'].iloc[-3]

## Creating Dashboard

Continue at main.py

In [15]:
# app = dash.Dash(__name__, meta_tags=[{"name": "viewport", "content": "width=device-width"}])

In [16]:
# app.layout = html.Div([
#     html.Div([
#         html.Div([
#             html.Img(src=app.get_asset_url('OIP.jpeg'),
#                      id='corona-image',
#                      style={
#                          'height':'60px',
#                          'width': 'auto',
#                          'margin-bottom':'25px',
#                      })
#         ])
#     ])
# ])

In [17]:
# if __name__ == '__main__':
#     app.run_server()